# Callibration

- input   0 0 1 0 0 2 2 2
- output  0 0 1 2 2 2 2 2

In [1]:
import sys

DATASET_PATH = ('../../../feedback-prize-2021')

sys.path.insert(0, '../codes/new_transformers_branch/transformers/src')
sys.path.append('../codes')
sys.path.append('..')

In [2]:
import os
import os.path as osp

import re
import pickle
import random
import easydict
import copy

from glob import glob
from tqdm.auto import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from new_transformers import DebertaV2TokenizerFast

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader

## Data

In [3]:
import h5py

data = h5py.File(f'../data_file/deberta_spm_data_v2.h5py')
labels = data[f'cbio_labels']
labels = np.argmax(labels, -1)
labels = labels[:-1]

In [4]:
token_n_list = []
for num_tokens in data['num_tokens']:
    token_n_list.append(num_tokens[0])
    
token_n_list = token_n_list[:-1]

## Augmentation

### Replacing category randomly by continuous 0 (length is 1 - 5)

In [5]:
test = copy.deepcopy(labels[0])
token_n = token_n_list[0]

# padding
test[token_n:] = -1

# place where zero class will be assigned
zero_n = random.randint(1, 5)
zero_start_i = random.randint(1, token_n - 1)
zero_end_i = min(zero_start_i + zero_n, token_n - 1)

# place that will be printed
print_start_i = max(zero_start_i - 10, 0)
print_end_i = min(zero_end_i + 10, token_n - 1)

print(f'0 category will be added at {zero_start_i} index with length {zero_n}')
print(f'before {test[print_start_i:print_end_i]}')

# augmentation
test[zero_start_i:zero_end_i] = 0

print(f'after  {test[print_start_i:print_end_i]}')

0 category will be added at 232 index with length 1
before [10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10]
after  [10 10 10 10 10 10 10 10 10 10  0 10 10 10 10 10 10 10 10 10 10]


### Replacing category randomly by any category (length is 30 - 100)

In [6]:
test = copy.deepcopy(labels[0])
token_n = token_n_list[0]

# padding
test[token_n:] = -1

# place where zero class will be assigned
replace_n = random.randint(30, 100)
replace_idx = np.random.choice(range(1, token_n - 1), size=replace_n, replace=False)
replace_cats = np.random.choice(range(0, 15), size=replace_n, replace=True)

# place that will be printed
print_start_i = max(replace_idx[0] - 10, 0)
print_end_i = min(replace_idx[0] + 10, token_n - 1)

print(f'category replacement will be happend at {replace_n} times')
print(f'showing at index {print_start_i}')
print(f'before {test[print_start_i:print_end_i]}')

# augmentation
test[replace_idx] = replace_cats

print(f'after  {test[print_start_i:print_end_i]}')

category replacement will be happend at 81 times
showing at index 149
before [6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
after  [ 6  6 12  6  6  6  6  6  5  1 12  6  8  6  6  2  6 10  8  6]


### Replace starting category as even number (ex: 122 -> 222)

In [7]:
test = copy.deepcopy(labels[0])
token_n = token_n_list[0]

# padding
test[token_n:] = -1

# place where odd category exist
odd_category = [1, 3, 5, 7, 9, 11, 13]

odd_cat_idx = [i for i, category in enumerate(test) if category in odd_category]
replace_n = random.randint(1, len(odd_cat_idx))
odd_cat_idx = np.random.choice(odd_cat_idx, size=replace_n, replace=False)

# place that will be printed
print_start_i = max(odd_cat_idx[0] - 5, 0)
print_end_i = min(odd_cat_idx[0] + 5, token_n - 1)

print(f'total {replace_n} will be replaced')
print(f'example: category {test[odd_cat_idx[0]]} will be replaced to {test[odd_cat_idx[0]] + 1}')
print(f'before {test[print_start_i:print_end_i]}')

# augmentation
test[odd_cat_idx] += 1

print(f'after  {test[print_start_i:print_end_i]}')

total 5 will be replaced
example: category 3 will be replaced to 4
before [2 2 2 2 0 3 4 4 4 4]
after  [2 2 2 2 0 4 4 4 4 4]


### Replace starting category to different category (ex: 122 -> 322)

In [8]:
test = copy.deepcopy(labels[0])
token_n = token_n_list[0]

# padding
test[token_n:] = -1

# place where odd category exist
odd_category = [1, 3, 5, 7, 9, 11, 13]
odd_cat_idx = [i for i, category in enumerate(test) if category in odd_category]

replace_n = random.randint(1, len(odd_cat_idx))
odd_cat_idx = np.random.choice(odd_cat_idx, size=replace_n, replace=False)

replace_cats = []
for odd_cat_i in odd_cat_idx:
    current_cat = test[odd_cat_i]
    odd_sample = copy.deepcopy(odd_category)
    odd_sample.remove(current_cat)
    replace_cats.append(random.choice(odd_sample))
    
replace_cats = np.array(replace_cats)

# place that will be printed
print_start_i = max(odd_cat_idx[0] - 5, 0)
print_end_i = min(odd_cat_idx[0] + 5, token_n - 1)

print(f'total {replace_n} will be replaced')
print(f'category {test[odd_cat_idx[0]]} will be replaced to {replace_cats[0]}')
print(f'before {test[print_start_i:print_end_i]}')

# augmentation
test[odd_cat_idx] = replace_cats

print(f'after  {test[print_start_i:print_end_i]}')

total 3 will be replaced
category 3 will be replaced to 11
before [2 2 2 2 0 3 4 4 4 4]
after  [ 2  2  2  2  0 11  4  4  4  4]


## Dataset

In [9]:
class CategoryDataset(torch.utils.data.Dataset):
    def __init__(self, labels, token_n_list):
        self.labels = labels
        self.token_n_list = token_n_list
        
        self.odd_category = [1, 3, 5, 7, 9, 11, 13]

    def replace_with_zero(self, label, token_n):
        """"""
        zero_n = random.randint(1, 5)
        zero_start_i = random.randint(1, token_n - 1)
        zero_end_i = min(zero_start_i + zero_n, token_n - 1)
        
        label[zero_start_i:zero_end_i] = 0
        
        return label
    
    def b_to_i_label(self, label, token_n):
        """"""
        odd_cat_idx = [i for i, category in enumerate(label) if category in self.odd_category]
        if len(odd_cat_idx) == 0:
            return label

        replace_n = random.randint(1, len(odd_cat_idx))
        odd_cat_idx = np.random.choice(odd_cat_idx, size=replace_n, replace=False)

        label[odd_cat_idx] += 1
        
        return label
    
    def replace_b_label(self, label, token_n):
        """"""
        odd_cat_idx = [i for i, category in enumerate(label) if category in self.odd_category]
        if len(odd_cat_idx) == 0:
            return label

        replace_n = random.randint(1, len(odd_cat_idx))
        odd_cat_idx = np.random.choice(odd_cat_idx, size=replace_n, replace=False)

        replace_cats = []
        for odd_cat_i in odd_cat_idx:
            current_cat = label[odd_cat_i]
            odd_sample = copy.deepcopy(self.odd_category)
            odd_sample.remove(current_cat)
            replace_cats.append(random.choice(odd_sample))

        replace_cats = np.array(replace_cats)

        # augmentation
        label[odd_cat_idx] = replace_cats
        
        return label
    
    def replace_with_random_label(self, label, token_n):
        """"""
        replace_n = random.randint(30, 100)
        replace_idx = np.random.choice(range(1, token_n - 1), size=replace_n, replace=False)
        replace_cats = np.random.choice(range(0, 15), size=replace_n, replace=True)

        label[replace_idx] = replace_cats

        return label
        
    def __getitem__(self, idx):
        targets = copy.deepcopy(self.labels[idx])
        inputs = copy.deepcopy(targets)
        token_n = self.token_n_list[idx]
        
        # padding
        inputs[token_n:] = 15
        targets[token_n:] = 15

        # augmentation
        inputs = self.replace_with_zero(inputs, token_n)
        inputs = self.b_to_i_label(inputs, token_n)
        inputs = self.replace_b_label(inputs, token_n)
        inputs = self.replace_with_random_label(inputs, token_n)
        
        return inputs, targets

    def __len__(self):
        return len(self.labels)

## Model

In [62]:
class Callibration(nn.Module):
    def __init__(self,
                 hidden_size=128,
                 n_layers=2,
                 device='cuda:0'):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.device = device

        self.embedding = nn.Embedding(16, hidden_size, padding_idx=15)
        self.lstm = nn.LSTM(hidden_size,
                            hidden_size,
                            n_layers,
                            bidirectional=True,
                            batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, 16)
        
    def init_hidden(self, batch_size):
        h = torch.zeros(self.n_layers * 2, batch_size, self.hidden_size)
        h = h.to(self.device)

        c = torch.zeros(self.n_layers * 2, batch_size, self.hidden_size)
        c = c.to(self.device)

        return (h, c)

    def forward(self, x):
        batch_size, seq_len = x.size()
        out = self.embedding(x)
        
        hidden = self.init_hidden(batch_size)
        out, hidden = self.lstm(out, hidden)
        
        out = out.contiguous().view(batch_size, -1, self.hidden_size * 2)
        out = self.fc(out)
        
        return out

## Train

In [31]:
batch_size = 32
device = 'cuda:0'

In [32]:
train_dataset = CategoryDataset(labels, token_n_list)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [63]:
model = Callibration().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    losses = []
    for step, (inputs, targets) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        batch_size, seq_len = inputs.size()
        
        preds = model(inputs)
        preds = preds.view(batch_size * seq_len, 16)
        targets = targets.view(-1)
        
        loss = criterion(preds, targets)
        losses.append(loss.item())
        

        if (step + 1) % 20 == 0:
            print(f"[ TRAIN ] epoch {epoch + 1} loss: {np.array(losses).mean():.4f}")
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  0%|          | 0/488 [00:00<?, ?it/s]

[ TRAIN ] epoch 1 loss: 1.2471
[ TRAIN ] epoch 1 loss: 0.6721
[ TRAIN ] epoch 1 loss: 0.4638
[ TRAIN ] epoch 1 loss: 0.3554
[ TRAIN ] epoch 1 loss: 0.2892
[ TRAIN ] epoch 1 loss: 0.2444
[ TRAIN ] epoch 1 loss: 0.2122
[ TRAIN ] epoch 1 loss: 0.1879
[ TRAIN ] epoch 1 loss: 0.1689
[ TRAIN ] epoch 1 loss: 0.1535
[ TRAIN ] epoch 1 loss: 0.1409
[ TRAIN ] epoch 1 loss: 0.1303
[ TRAIN ] epoch 1 loss: 0.1213
[ TRAIN ] epoch 1 loss: 0.1135
[ TRAIN ] epoch 1 loss: 0.1068
[ TRAIN ] epoch 1 loss: 0.1008
[ TRAIN ] epoch 1 loss: 0.0956
[ TRAIN ] epoch 1 loss: 0.0909
[ TRAIN ] epoch 1 loss: 0.0867
[ TRAIN ] epoch 1 loss: 0.0829
[ TRAIN ] epoch 1 loss: 0.0795
[ TRAIN ] epoch 1 loss: 0.0764
[ TRAIN ] epoch 1 loss: 0.0735
[ TRAIN ] epoch 1 loss: 0.0708


  0%|          | 0/488 [00:00<?, ?it/s]

[ TRAIN ] epoch 2 loss: 0.0100
[ TRAIN ] epoch 2 loss: 0.0100
[ TRAIN ] epoch 2 loss: 0.0098
[ TRAIN ] epoch 2 loss: 0.0097
[ TRAIN ] epoch 2 loss: 0.0096
[ TRAIN ] epoch 2 loss: 0.0096
[ TRAIN ] epoch 2 loss: 0.0095
[ TRAIN ] epoch 2 loss: 0.0094
[ TRAIN ] epoch 2 loss: 0.0093
[ TRAIN ] epoch 2 loss: 0.0093
[ TRAIN ] epoch 2 loss: 0.0093
[ TRAIN ] epoch 2 loss: 0.0092
[ TRAIN ] epoch 2 loss: 0.0092
[ TRAIN ] epoch 2 loss: 0.0091
[ TRAIN ] epoch 2 loss: 0.0091
[ TRAIN ] epoch 2 loss: 0.0090
[ TRAIN ] epoch 2 loss: 0.0090
[ TRAIN ] epoch 2 loss: 0.0090
[ TRAIN ] epoch 2 loss: 0.0090
[ TRAIN ] epoch 2 loss: 0.0089
[ TRAIN ] epoch 2 loss: 0.0089
[ TRAIN ] epoch 2 loss: 0.0089
[ TRAIN ] epoch 2 loss: 0.0088
[ TRAIN ] epoch 2 loss: 0.0088


  0%|          | 0/488 [00:00<?, ?it/s]

[ TRAIN ] epoch 3 loss: 0.0084
[ TRAIN ] epoch 3 loss: 0.0082
[ TRAIN ] epoch 3 loss: 0.0081
[ TRAIN ] epoch 3 loss: 0.0080
[ TRAIN ] epoch 3 loss: 0.0080
[ TRAIN ] epoch 3 loss: 0.0080
[ TRAIN ] epoch 3 loss: 0.0080
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079
[ TRAIN ] epoch 3 loss: 0.0079


  0%|          | 0/488 [00:00<?, ?it/s]

[ TRAIN ] epoch 4 loss: 0.0078
[ TRAIN ] epoch 4 loss: 0.0076
[ TRAIN ] epoch 4 loss: 0.0075
[ TRAIN ] epoch 4 loss: 0.0075


KeyboardInterrupt: 

In [92]:
token_n = 15
inputs = torch.zeros(2048)

inputs[token_n:] = 15

inputs[:token_n] = torch.Tensor([0, 1, 2, 2, 0, 4, 0, 4, 3, 5, 0, 0, 1, 4, 0])
inputs = inputs.long().to(device).unsqueeze(0)

output = model(inputs)
cal_output = output.argmax(-1)

In [93]:
inputs[:, :15]

tensor([[0, 1, 2, 2, 0, 4, 0, 4, 3, 5, 0, 0, 1, 4, 0]], device='cuda:0')

In [94]:
cal_output[:, :15]

tensor([[0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0]], device='cuda:0')

In [95]:
torch.save(model.state_dict(), '../result/callibration.pt')